In [1]:
from config_path import add_to_sys_path
add_to_sys_path()  # Call the function to add path

import numpy as np
from sympy.physics.wigner import wigner_3j,wigner_6j

In [17]:
def BRFF(BRJJ,Jg,Fg,Fe):
    Je = 1/2
    i = 1/2
    BR = wigner_6j(Je, Fe, i, Fg, Jg, 1)**2 * (2*Fg + 1)*(2*Je + 1) * BRJJ
    return float(BR)

In [39]:
BRJJ = [1/3,1/6,1/2]
Jg = [1/2,3/2,3/2]
Fe = [(0,1),(0,1),(0,1)]
Fg = [(0,1),(1,2),(1,2)]

for i,(_brjj,_jg) in enumerate(zip(BRJJ,Jg)):
    for _fg in Fg[i]:
        for _fe in Fe[i]:
            _brff = BRFF(_brjj, _jg,_fg,_fe)
            print('Decay from Fe={Fe} -> Fg={Fg}, Jg={Jg} is {br}'.format(Fe=_fe,Fg=_fg,Jg=_jg,br=np.round(_brff,3)))
        avg = np.round((3*BRFF(_brjj, _jg,_fg,1)+BRFF(_brjj, _jg,_fg,0))/4,3)
        print('Avg decay is to Fg={Fg},Jg={Jg} is {avg}'.format(Fg=_fg,Jg=_jg,avg=avg))

Decay from Fe=0 -> Fg=0, Jg=0.5 is 0.0
Decay from Fe=1 -> Fg=0, Jg=0.5 is 0.111
Avg decay is to Fg=0,Jg=0.5 is 0.083
Decay from Fe=0 -> Fg=1, Jg=0.5 is 0.333
Decay from Fe=1 -> Fg=1, Jg=0.5 is 0.222
Avg decay is to Fg=1,Jg=0.5 is 0.25
Decay from Fe=0 -> Fg=1, Jg=1.5 is 0.167
Decay from Fe=1 -> Fg=1, Jg=1.5 is 0.028
Avg decay is to Fg=1,Jg=1.5 is 0.062
Decay from Fe=0 -> Fg=2, Jg=1.5 is 0.0
Decay from Fe=1 -> Fg=2, Jg=1.5 is 0.139
Avg decay is to Fg=2,Jg=1.5 is 0.104
Decay from Fe=0 -> Fg=1, Jg=1.5 is 0.5
Decay from Fe=1 -> Fg=1, Jg=1.5 is 0.083
Avg decay is to Fg=1,Jg=1.5 is 0.188
Decay from Fe=0 -> Fg=2, Jg=1.5 is 0.0
Decay from Fe=1 -> Fg=2, Jg=1.5 is 0.417
Avg decay is to Fg=2,Jg=1.5 is 0.312


In [33]:
for br0,br1 in [(0,0.186),(0.558,0.372),(0.007)]

0.0405

In [34]:
0.027+0.081+0.310+0.516+0.025+0.040

0.999

In [79]:
def kronecker(a,b):         # Kronecker delta function
    if a==b:
        return 1
    else:
        return 0

def b2a_matrix(a,b,S=1/2):
    if not kronecker(a['K'],b['K'])*kronecker(a['J'],b['J'])*kronecker(a['F'],b['F'])*kronecker(a['M'],b['M']):
        return 0
    else:
        if 'F1' in b.keys():
            if not kronecker(a['F1'],b['F1']):
                return 0
        return (-1)**(b['N']-S+a['P'])*np.sqrt(2*b['N']+1)*wigner_3j(a['J'],S,b['N'],a['P'],-a['Sigma'],-a['K'])
    
def convert_abBJ(input_qnumbers,output_qnumbers,S=1/2):
    input_keys = list(input_qnumbers)
    output_keys = list(output_qnumbers)
    input_size = len(input_qnumbers[input_keys[0]])
    output_size = len(output_qnumbers[output_keys[0]])
    basis_matrix = np.zeros((output_size,input_size))
    for i in range(output_size):
        for j in range(input_size):
            if 'N' in input_keys: #Convert case (b) to (a)
                a_qnumbers = {q:output_qnumbers[q][i] for q in output_keys}
                b_qnumbers = {q:input_qnumbers[q][j] for q in input_keys}
            else:
                b_qnumbers = {q:output_qnumbers[q][i] for q in output_keys}
                a_qnumbers = {q:input_qnumbers[q][j] for q in input_keys}
            basis_matrix[i,j] = b2a_matrix(a_qnumbers,b_qnumbers,S=S)
    return basis_matrix

def q_numbers_even_bBJ(N_range,K_mag,S=1/2,I_list=[0,1/2],M_values='all',M_range=[]):
    IM=I_list[0]
    iH = I_list[-1]
    Nmin,Nmax=N_range[0],N_range[-1]
    K_mag = abs(K_mag)
    if Nmin<K_mag:
        print('Nmin must be >= |K|')
        Nmin=abs(K_mag)
    q_str = ['K','N','J','F','M']
    I = max(IM,iH)
    q_numbers = {}
    for q in q_str:
        q_numbers[q] = []
    for N in np.arange(Nmin,Nmax+1,1):
        for J in np.arange(abs(N-S),abs(N+S)+1,1):
            for F in np.arange(abs(J-I),abs(J+I)+1,1):
                if M_values=='none':
                    for K in {True:[0], False:[-K_mag,K_mag]}[K_mag==0]:
                        M=abs(F)%1
                        values = [K,N,J,F,M]
                        for q,val in zip(q_str,values):
                            q_numbers[q].append(val+0)    #looks weird but adding 0 converts -0 to 0
                else:
                    if M_values=='all' or M_values=='custom':
                        Mmin = -F
                    elif M_values=='pos':
                        Mmin = abs(F) % 1
                    for M in np.arange(Mmin,F+1,1):
                        if (M_values=='custom' and M in M_range) or (M_values=='all') or (M_values=='pos'):
                            for K in {True:[0], False:[-K_mag,K_mag]}[K_mag==0]:
                                values = [K,N,J,F,M]
                                for q,val in zip(q_str,values):
                                    q_numbers[q].append(val+0)    #looks weird but adding 0 converts -0 to 0
                        elif M_values=='custom' and M not in M_range:
                            continue
    return q_numbers

def q_numbers_even_aBJ(N_range,K_mag=1,S=1/2,I_list=[0,1/2],M_values='all',P_values=[1/2],M_range=[]):
    IM=I_list[0]
    iH = I_list[-1]
    Nmin,Nmax=N_range[0],N_range[-1]
    Jmin = abs(Nmin-S)
    Jmax = abs(Nmax+S)
    K_mag = abs(K_mag)
    if Nmin<K_mag:
        print('Nmin must be >= |K|')
        Nmin=abs(K_mag)
    q_str = ['K','Sigma','P','J','F','M']
    q_numbers = {}
    for q in q_str:
        q_numbers[q] = []
    I = max(IM,iH)
    for J in np.arange(Jmin,Jmax+1,1):
        for F in np.arange(abs(J-I),abs(J+I)+1,1):
            if M_values=='none':
                for Sigma in np.arange(-abs(S),abs(S)+1,1):
                    for K in {True:[0], False:[-K_mag,K_mag]}[K_mag==0]:
                        P=K+Sigma
                        if abs(P) not in P_values:
                            continue
                        elif J < abs(P):
                            continue
                        else:
                            M=abs(F)%1
                            values = [K,Sigma,P,J,F,M]
                        for q,val in zip(q_str,values):
                            q_numbers[q].append(val+0)    #looks weird but adding 0 converts -0 to 0
            else:
                if M_values=='all' or M_values=='custom':
                    Mmin = -F
                elif M_values=='pos':
                    Mmin = abs(F) % 1
                for M in np.arange(Mmin,F+1,1):
                    if (M_values=='custom' and M in M_range) or (M_values=='all') or (M_values=='pos'):
                        for Sigma in np.arange(-abs(S),abs(S)+1,1):
                            for K in {True:[0], False:[-K_mag,K_mag]}[K_mag==0]:
                                P=K+Sigma
                                if abs(P) not in P_values:
                                    continue
                                elif J < abs(P):
                                    continue
                                else:
                                    values = [K,Sigma,P,J,F,M]
                                for q,val in zip(q_str,values):
                                    q_numbers[q].append(val+0)    #looks weird but adding 0 converts -0 to 0
                    elif M_values=='custom' and M not in M_range:
                        continue
    return q_numbers

def write_state(vector,qn,case):
    for index in np.nonzero(vector)[0]:
        v={q:qn[q][index] for q in qn}
        coeff = vector[index]
        if case == 'bBS':
            print(' {} |K={},N={},G={},F1={},F={},M={}> \n'.format(coeff,v['K'],v['N'],v['G'],v['F1'],v['F'],v['M']))
        elif case == 'bBJ':
            print(' {} |K={},N={},J={},F={},M={}> \n'.format(coeff,v['K'],v['N'],v['J'],v['F'],v['M']))
        elif case == 'aBJ':
            print(' {} |K={},\u03A3={},P={},J={},F={},M={}> \n'.format(coeff,v['K'],v['Sigma'],v['P'],v['J'],v['F'],v['M']))

In [92]:
qb = q_numbers_even_bBJ([0,1,2],0,M_values='none',I_list=[0,0])
qa = q_numbers_even_aBJ([0,1,2],K_mag=0,M_values='none',I_list=[0,0])
na = len(qa['J'])
nb = len(qb['N'])
print(qa)
print()
print(qb)

{'K': [0, 0, 0, 0, 0, 0], 'Sigma': [-0.5, 0.5, -0.5, 0.5, -0.5, 0.5], 'P': [-0.5, 0.5, -0.5, 0.5, -0.5, 0.5], 'J': [0.5, 0.5, 1.5, 1.5, 2.5, 2.5], 'F': [0.5, 0.5, 1.5, 1.5, 2.5, 2.5], 'M': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]}

{'K': [0, 0, 0, 0, 0], 'N': [0, 1, 1, 2, 2], 'J': [0.5, 0.5, 1.5, 1.5, 2.5], 'F': [0.5, 0.5, 1.5, 1.5, 2.5], 'M': [0.5, 0.5, 0.5, 0.5, 0.5]}


In [119]:
vb = np.zeros(nb)
vb[0]=1

In [120]:
b2a=convert_abBJ(qb,qa,S=1/2)

In [121]:
write_state(vb,qb,'bBJ')

 1.0 |K=0,N=0,J=0.5,F=0.5,M=0.5> 



In [122]:
write_state(b2a@vb,qa,'aBJ')

 0.7071067811865476 |K=0,Σ=-0.5,P=-0.5,J=0.5,F=0.5,M=0.5> 

 0.7071067811865476 |K=0,Σ=0.5,P=0.5,J=0.5,F=0.5,M=0.5> 



In [77]:
b2a=convert_abBJ(qb,qa,S=1/2)

In [64]:
a={}
b={}
a['J']=1/2
a['F']=1/2
a['M']=0
a['K']=0
a['P']=-1/2
a['Sigma']=a['P']
b['N']=1
b['J']=1/2
b['K']=0
b['F']=1/2
b['M']=0

In [65]:
b2a_matrix(a,b)

0.288675134594813*sqrt(6)

In [132]:
def q_numbers_A010(J,l=1,L=1,S=1/2,P_values=[1/2,3/2,5/2]):
    q_str = ['K','Lambda','l','P','J','Par']
    q_numbers = {}
    for q in q_str:
        q_numbers[q] = []
    for K in [abs(L-l),abs(L+l)]:
        for P in [abs(K-S),abs(K+S)]:
            if abs(P) not in P_values:
                continue
            elif J < abs(P):
                continue
            else:
                for parity in [1,-1]:
                    values = [K,L,l,P,J,parity]
                    for q,val in zip(q_str,values):
                            q_numbers[q].append(val+0)
    return q_numbers

In [133]:
q_numbers_A010(1/2)

{'K': [0, 0, 0, 0],
 'Lambda': [1, 1, 1, 1],
 'l': [1, 1, 1, 1],
 'P': [0.5, 0.5, 0.5, 0.5],
 'J': [0.5, 0.5, 0.5, 0.5],
 'Par': [1, -1, 1, -1]}

In [ ]:
params_A010={
    'Ts':,
    'Td':,
    'Bd':,
    'Bs-':,
    'Bs+':,
    'Dd':0,
    'Ds-':0,
    'Ds+':0,
    'A':,
    'AD':0,
    'e':,
    'gs-':,
    'gs+':,
    'gsD-':,
    'gsD+':,
    'qe':,
    'qv':,
    'pe+2qe':,
    'w2':,
}

def A010(J,parity,params):
    n=4
    Ts=0
    H = np.zeros((n,n))
    z=np.sqrt(J+1/2)
    Ad = params['A']*(1-3/4*params['e']**2)
    As = params['A']*(1-1/4*params['e']**2)
    AdD = params['AD']*(1-3/4*params['e']**2)
    AsD = params['AD']*(1-1/4*params['e']**2)
    H[0,0] = params['Ts']-params['Td']+params['Bd']*(z-6)-params['Dd'] (z**2-11*z+32)+1/2*(Ad+AdD*(z-6))
    H[0,1] = -params['Bd']*np.sqrt(z-4)+2*params['Dd']*(z-4)**(3/2)
    H[1,0] = H[0,1]
    H[1,1] = params['Ts'] - params['Td'] + params['Bd']*(z-2) - params['Dd']*z*(z-3)-1/2*(Ad+AdD*(z-2))
    H[0,2] = 1/2*params['qv']*(z**2-5*z+4)**(1/2)
    H[2,0] = H[0,2]
    H[1,2] = (z-1)**(1/2)*(parity*1/2*(params['qe']*z**(1/2)-params['qv']))
    H[2,1] = H[1,2]
    H[2,2] = params['Ts'] - 1/2*(As+AsD*z) - parity*1/2*params['gs-']*z**(1/2)+params['Bs-']*z-params['Ds-']*z*(z+1)-parity*params['gs-D']*z**(1/2)*(z+1+parity*z**(1/2))
    H[0,3] = parity*params['qe']*(z**2 - 5*z + 4)**(1/2)
    H[3,0] = H[0,3]
    H[1,3] = (z-1)**(1/2)*(-parity/2*params['pe+2qe']+1/2*params['qv']*z**(1/2))
    H[3,1] = H[1,3]
    H[2,3] = -1/2*(params['Bs-']+params['Bs+'])*z**(1/2) + (params['Ds-']+params['Ds+'])*z**(3/2)+params['e']*params['w2']
    H[3,2] = H[2,3]
    H[3,3] = params['Ts'] - 1/2*(As+AsD*z)+parity*params['gs+']*z**(1/2)+params['Bs+']*z - params['Ds+']*z*(z+1)+parity/2*params['gs+d']*z**(1/2)*(z+1-parity*2*z**(1/2))
    return H